
grafana llm service?

ROUTE = /v1/chat/completions

METHOD = post


In [ ]:
const { Remarkable } = require('remarkable')
const md = new Remarkable({ html: true, xhtmlOut: true, breaks: true })
const crypto = require('crypto');

function generateHash(data, algorithm = 'sha256', encoding = 'hex') {
  const hash = crypto.createHash(algorithm);
  hash.update(data);
  return hash.digest(encoding);
}

let TIMEOUT = 15 * 1000
let initialChatHistory
let session
let hashes = [

]

async function grafanaRequest(req, res) {
  if (!session) {
    const { createSession } = await importer.import('llama vision')
    session = await createSession(process.env.DEFAULT_MODEL || 'gemma-3-27b-it-Q6_K.gguf', req.body.messages[0].content)
    initialChatHistory = session.getChatHistory();
  }

  let start = Date.now()

  if (req.body.messages && req.body.messages.length) {
    req.body.message = req.body.messages[req.body.messages.length - 1].content
  }

  res.set('Connection', 'keep-alive');
  res.set('Keep-Alive', 'timeout=' + (TIMEOUT / 1000) + ', max=' + (TIMEOUT / 1000))

  let hash = generateHash(req.body.content || req.body.message)
  if (typeof hashes[hash] != 'undefined') {
    if (hashes[hash].status && hashes[hash].status == 'pending') {
      await new Promise(resolve => setTimeout(resolve, TIMEOUT - 5)) // make client wait for requesting
      return res.json(hashes[hash])
    } else {
      return res.json(hashes[hash])
    }
  }
  console.log('User: ' + (req.body.content || req.body.message))
  let result = await session.prompt(req.body.content || req.body.message, {
    //temperature: temperature,
    onTextChunk: function (text) {
      process.stdout.write(text)
      if (typeof hashes[hash] == 'undefined' && start + TIMEOUT - 5 < Date.now()) {
        hashes[hash] = { "id": "resp_" + hash, "status": "pending" }
        res.json(hashes[hash])
      }
    }
  })

  console.log('AI: ' + result)
  session.setChatHistory(initialChatHistory)

  let code = result.matchAll(/```(json|bash|javascript|code|markdown)*\n[\s\S]*?\n*```/gi)
  if (code) {
    result = result.replaceAll(/^```(json|bash|javascript|code|markdown)\n*|\n*```\s*$/gi, '')
  }

  if (typeof hashes[hash] != 'undefined') {
    // request already returned data?
    if (hashes[hash].status && hashes[hash].status == 'pending') {
      hashes[hash] = { content: result, object: result }
      return
      // previous request?
    } else {
      hashes[hash] = { content: result, object: result }
      return res.json({ content: result, object: result })
    }
  }

  if (req.headers.accept == 'application/json') {
    return res.json({ content: result, object: result });
  }
  const mdHtml = md.render(result)
  return res.send(mdHtml)
}

module.exports = grafanaRequest



show grafana metrics?

ROUTE = /metrics


In [ ]:

function printMetrics(req, res, next) {
    res.header('Cache-Control', 'no-store, no-cache, must-revalidate, proxy-revalidate');
    res.header('Pragma', 'no-cache');
    res.header('Expires', '0');
    res.setHeader('content-type', 'text/plain')
    res.send(console.metrics())
}

module.exports = printMetrics


TODO: make a metrics for prometheus and logging for loki output

spread the console format like [service][debug] message throughout the notebooks

log by replacing console.log in services by importing constants from logging service




start a bunch of grafana services?


In [ ]:

const {spawn, spawnSync} = require("child_process");
const path = require('path')

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE

const COMMANDS = [
  ['./prometheus', '--config.file=prometheus.yml', '--web.listen-address=0.0.0.0:9292', '--web.enable-remote-write-receiver'],
  ['grafana-agent', '--config.file=grafana-agent.yaml'],
  ['/opt/homebrew/Cellar/loki/3.4.3/bin/loki', '--config.file=loki-local-config.yaml'],
  // https://grafana.com/docs/alloy/latest/configure/macos/#configure-the-alloy-service
  ['docker', 'compose', 'up', '--build'],
  //['webpack', '-w', '-c', './.config/webpack/webpack.config.ts', '--env', 'development'],
  ['npm', 'run', 'dev'],
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'grafana llm service'],
]

const ENVIRONMENTS = [
  {
    CWD: path.join(HOMEPATH, 'prometheus-3.3.0-rc.0.darwin-arm64'),
    SERVICE_NAME: 'prometheus'
  },
  {
    CWD: path.join(HOMEPATH, 'grafana', 'apparentli-app'),
    SERVICE_NAME: 'grafana-agent'
  },
  {
    CWD: path.join(HOMEPATH, 'grafana', 'apparentli-app'),
    SERVICE_NAME: 'loki'
  },
  {
    CWD: path.join(HOMEPATH, 'grafana', 'apparentli-app'),
    SERVICE_NAME: 'grafana-docker'
  },
  {
    CWD: path.join(HOMEPATH, 'grafana', 'apparentli-app'),
    SERVICE_NAME: 'grafana-dev'
  },
  {
    CWD: path.join(HOMEPATH, 'jupyter_ops'),
    SERVICE_NAME: 'grafana-llm',
    CHAT_PORT: 9393
  },
]

function launchGrafana(botIndex = 0) {

  if(parseInt(botIndex).toString() != botIndex.toString()) {
    return
  } else {
    botIndex = parseInt(botIndex)
  }

  if(typeof botIndex != 'number' && !botIndex) {
    return
  }

  let children = []

  for(let i = botIndex; i < ENVIRONMENTS.length; i++) {
    if(!ENVIRONMENTS[i] || !COMMANDS[i]) {
      continue
    }

    child = spawn(COMMANDS[i][0], COMMANDS[i].slice(1), {
      env: Object.assign({}, process.env, ENVIRONMENTS[i] ? ENVIRONMENTS[i] : {}),
      stdio: ['pipe', 'pipe', 'pipe'],
      //detached: true,
      //shell: true,
      cwd: ENVIRONMENTS[i].CWD || path.dirname(__dirname)
    })

    child.stdout.on('data', (data) => console.log(data.toString().split('\n').map(line => '[' + ENVIRONMENTS[i].SERVICE_NAME + ']' + line).join('\n')))
    child.stderr.on('data', (data) => console.error(data.toString().split('\n').map(line => '[' + ENVIRONMENTS[i].SERVICE_NAME + ']' + line).join('\n')))

    children.push(child)

    if(botIndex) {
      break
    }
  }

  process.on('SIGINT', function() {
    children.forEach(child => child.kill())
  })
}

module.exports = launchGrafana
